# 🎤 OpenWakeWord Model Trainer

Train custom wake word models for [openWakeWord](https://github.com/dscripka/openWakeWord).

## Features
- ✅ Train **multiple wake words** at once
- ✅ **Better parameter controls** with clear explanations
- ✅ **Quick test mode** for fast iteration
- ✅ **Progress logging** with clear status indicators
- ✅ **Google Drive integration** for reliable model saving
- ✅ Produces **ONNX models** (works with Home Assistant, Python, etc.)

## How to Use

**Note:** All steps must be executed in order!

1. **Step 1**: Test pronunciation - make sure your wake word sounds right
2. **Step 2**: Configure training parameters
3. **Step 3**: Download data (~15-20 min)
4. **Step 4**: Train model (~30-90 min depending on settings) - includes Google Drive option!
5. **Step 5**: Download your `.onnx` model file (backup if not using Google Drive)

**Tip:** Use GPU runtime for faster training: Runtime → Change runtime type → T4 GPU

In [13]:
# @title ## 🎧 Step 1: Test Wake Word Pronunciation { display-mode: "form" }
# @markdown **Test how your wake word will sound before training!**
# @markdown
# @markdown First run takes ~1-2 minutes to setup. Subsequent runs are fast.
# @markdown
# @markdown ### Pronunciation Tips
# @markdown - Use underscores for syllable breaks: `computer` → `khum_puter`
# @markdown - Spell phonetically: `jarvis` → `jar_viss`
# @markdown - Multi-word: `hey jarvis` → `hey_jar_viss`
# @markdown - Spell out numbers: `2` → `two`
# @markdown - Avoid punctuation except `?` and `!`

target_word = 'how_do_you_wanna_do_this!?' # @param {type:"string"}

import os
import sys
from IPython.display import Audio, display

# Setup TTS on first run
if not os.path.exists("./piper-sample-generator"):
    print("🔧 First run - setting up TTS engine (~1-2 minutes)...")
    !git clone https://github.com/rhasspy/piper-sample-generator
    !wget -q -O piper-sample-generator/models/en_US-libritts_r-medium.pt 'https://github.com/rhasspy/piper-sample-generator/releases/download/v2.0.0/en_US-libritts_r-medium.pt'
    !cd piper-sample-generator && git checkout 213d4d5
    !pip install -q piper-tts piper-phonemize-cross
    !pip install -q webrtcvad
    !pip install -q --force-reinstall 'torch==2.4.0' 'torchaudio==2.4.0' torchvision
    print("✅ TTS setup complete!\n")

if "piper-sample-generator/" not in sys.path:
    sys.path.append("piper-sample-generator/")

# Check for torch/torchaudio compatibility
try:
    import torchaudio
    torchaudio.load  # Test that it works
except (OSError, ImportError) as e:
    if "undefined symbol" in str(e) or "libtorchaudio" in str(e):
        print("⚠️ Torch/torchaudio version mismatch detected!")
        print("   This happens if Step 3 was run before Step 1.")
        print("   Please: Runtime → Restart session, then run Step 1 first.")
        raise RuntimeError("Please restart runtime and run Step 1 before Step 3")

# ============================================================
# FIX: Patch torch.load ONLY ONCE to avoid recursion error
# ============================================================
import torch

# Check if we've already patched torch.load (prevents RecursionError on re-run)
if not getattr(torch.load, '_oww_patched', False):
    _original_torch_load = torch.load

    def _patched_torch_load(*args, **kwargs):
        kwargs.setdefault('weights_only', False)
        return _original_torch_load(*args, **kwargs)

    # Mark it as patched so we don't do it again
    _patched_torch_load._oww_patched = True
    torch.load = _patched_torch_load

from generate_samples import generate_samples

def preview_wake_word(text):
    """Generate and play a sample of the wake word."""
    print(f"🎤 Generating audio for: '{text}'")
    generate_samples(
        text=text,
        max_samples=1,
        length_scales=[1.1],
        noise_scales=[0.7],
        noise_scale_ws=[0.7],
        output_dir='./',
        batch_size=1,
        auto_reduce_batch_size=True,
        file_names=["test_generation.wav"]
    )
    return Audio("test_generation.wav", autoplay=True)

print("\n▶️ Listen to your wake word:")
display(preview_wake_word(target_word))
print("\n💡 If it doesn't sound right, change the spelling above and run again!")
print("   Once satisfied, proceed to Step 2.")

DEBUG:generate_samples:Loading /home/stud/j/js490/repo/wakeword-howdoyouwanndothis/piper-sample-generator/models/en_US-libritts_r-medium.pt
INFO:generate_samples:Successfully loaded the model
DEBUG:generate_samples:CUDA available, using GPU
DEBUG:generate_samples:Batch 1/1 complete
INFO:generate_samples:Done



▶️ Listen to your wake word:
🎤 Generating audio for: 'how_do_you_wanna_do_this!?'



💡 If it doesn't sound right, change the spelling above and run again!
   Once satisfied, proceed to Step 2.


In [2]:
# @title ## ⚙️ Step 2: Training Configuration { display-mode: "form" }
# @markdown ### Wake Words
# @markdown Enter one or more wake words, separated by commas.
# @markdown Use the exact spelling that sounded best in Step 1!
# @markdown
# @markdown **Examples:** `hey_jar_viss` or `hey_jar_viss, oh_kay_computer`

wake_words = "how_do_you_wanna_do_this!?" # @param {type:"string"}

# @markdown ---
# @markdown ### Quick Test Mode
# @markdown Enable for faster training (~30 min) with lower quality. Good for testing!
quick_test_mode = True # @param {type:"boolean"}

# @markdown ---
# @markdown ### Training Parameters
# @markdown These are ignored if Quick Test Mode is enabled.
# @markdown
# @markdown | Parameter | Low | Default | High | Effect |
# @markdown |-----------|-----|---------|------|--------|
# @markdown | Examples | 5,000 | 25,000 | 50,000 | More = better quality, longer training |
# @markdown | Steps | 5,000 | 25,000 | 50,000 | More = better convergence, longer training |
# @markdown | False Activation Penalty | 500 | 1,500 | 3,000 | Higher = fewer false triggers, may miss quiet speech |

_number_of_examples = 25000 # @param {type:"slider", min:1000, max:50000, step:1000}
_number_of_training_steps = 25000 # @param {type:"slider", min:1000, max:50000, step:1000}
_false_activation_penalty = 1500 # @param {type:"slider", min:100, max:5000, step:100}

# @markdown ---
# @markdown ### Advanced Options
# @markdown
# @markdown **target_false_positives_per_hour** - How often model incorrectly triggers
# @markdown - `0.1` = ~1 false trigger every 10 hours (very strict)
# @markdown - `0.5` = ~1 false trigger every 2 hours (stricter)
# @markdown - `1.0` = ~1 false trigger per hour (permissive)

target_false_positives_per_hour = 0.8 # @param {type:"number"}

# @markdown **target_recall** - Percentage of real wake words detected (at evaluation threshold)
# @markdown - `0.5` = 50% detected (conservative, fewer false positives)
# @markdown - `0.7` = 70% detected (balanced)
# @markdown - `0.9` = 90% detected (sensitive, more false positives)

target_recall = 0.7 # @param {type:"number"}

# @markdown **layer_size** - Neural network hidden layer size (affects model size and accuracy)
# @markdown - `32` = ~15 KB model, fastest inference, good for simple single words
# @markdown - `64` = ~30 KB model, fast, balanced
# @markdown - `96` = ~50 KB model, better accuracy for multi-word phrases
# @markdown - `128` = ~75 KB model, best accuracy, slower inference

layer_size = 96 # @param [32, 64, 96, 128] {type:"raw"}

# Hidden defaults (not exposed in UI)
target_accuracy = 0.7  # Not configurable - doesn't significantly affect training

# ============================================================
# APPLY SETTINGS
# ============================================================

# Parse wake words
wake_word_list = [w.strip() for w in wake_words.split(',') if w.strip()]

if not wake_word_list:
    raise ValueError("❌ No wake words specified! Enter at least one wake word above.")

# Apply quick test mode
if quick_test_mode:
    number_of_examples = 5000
    number_of_training_steps = 5000
    false_activation_penalty = 500
    print("⚡ QUICK TEST MODE ENABLED")
    print("   Using reduced settings for faster training (~30 min)")
    print("   Model quality will be lower - for testing only!")
else:
    number_of_examples = _number_of_examples
    number_of_training_steps = _number_of_training_steps
    false_activation_penalty = _false_activation_penalty

print(f"\n{'='*50}")
print(f"📋 TRAINING CONFIGURATION")
print(f"{'='*50}")
print(f"\n🎯 Wake words to train: {wake_word_list}")
print(f"\n📊 Training parameters:")
print(f"   • Examples per word: {number_of_examples:,}")
print(f"   • Training steps: {number_of_training_steps:,}")
print(f"   • False activation penalty: {false_activation_penalty}")
print(f"\n📈 Evaluation targets:")
print(f"   • Target FP/hour: {target_false_positives_per_hour}")
print(f"   • Target recall: {target_recall*100:.0f}%")
print(f"\n🧠 Model architecture:")
print(f"   • Layer size: {layer_size} neurons")
print(f"\n✅ Configuration saved. Proceed to Step 3.")

⚡ QUICK TEST MODE ENABLED
   Using reduced settings for faster training (~30 min)
   Model quality will be lower - for testing only!

📋 TRAINING CONFIGURATION

🎯 Wake words to train: ['how_do_you_wanna_do_this!?']

📊 Training parameters:
   • Examples per word: 5,000
   • Training steps: 5,000
   • False activation penalty: 500

📈 Evaluation targets:
   • Target FP/hour: 0.8
   • Target recall: 70%

🧠 Model architecture:
   • Layer size: 96 neurons

✅ Configuration saved. Proceed to Step 3.


In [3]:
# @title ## 📦 Step 3: Download Data & Setup Environment { display-mode: "form" }
# @markdown This downloads all required data and installs dependencies.
# @markdown
# @markdown **Time:** ~15-20 minutes (mostly downloading)
# @markdown
# @markdown **What gets downloaded:**
# @markdown - Pre-computed audio features (~17 GB) - for negative examples
# @markdown - Validation features (~176 MB) - for false positive testing
# @markdown - Room impulse responses - for reverb augmentation
# @markdown - Background audio (music/noise) - for augmentation
# @markdown
# @markdown ⚠️ **License Note:** Data has mixed licenses. Models trained here are for **non-commercial personal use only**.

import locale
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

import os
import sys
from pathlib import Path

print("="*60)
print("📦 STEP 3: ENVIRONMENT SETUP & DATA DOWNLOAD")
print("="*60)

# ============================================================
# 3.1 INSTALL DEPENDENCIES (must happen before scipy/numpy imports!)
# ============================================================
print("\n🔧 Installing dependencies...")

# Unload any cached numpy/scipy modules to avoid version conflicts
mods_to_remove = [k for k in sys.modules.keys() if k.startswith(('numpy', 'scipy'))]
for mod in mods_to_remove:
    del sys.modules[mod]

# Fix numpy/scipy compatibility FIRST
!pip install -q --force-reinstall 'numpy==1.26.4' 'scipy==1.13.1'

!git clone -q https://github.com/dscripka/openwakeword 2>/dev/null || echo "openwakeword already cloned"
!pip install -q -e ./openwakeword --no-deps

# Core dependencies
!pip install -q mutagen==1.47.0
!pip install -q torchinfo==1.8.0
!pip install -q torchmetrics==1.2.0
!pip install -q speechbrain==0.5.14
!pip install -q audiomentations==0.33.0
!pip install -q torch-audiomentations==0.11.0
!pip install -q acoustics==0.2.6
!pip install -q onnxruntime==1.22.1 ai_edge_litert==1.4.0 onnxsim
!pip install -q onnx onnx_graphsurgeon sng4onnx
!pip install -q onnx_tf tensorflow 2>/dev/null || true  # Prevents train.py crash
!pip install -q pronouncing==0.2.0
!pip install -q datasets==2.14.6
!pip install -q deep-phonemizer==0.0.19

print("✅ Dependencies installed")

# ============================================================
# NOW import scipy/numpy after installation (fresh import)
# ============================================================
import numpy as np
import scipy.io.wavfile
from tqdm.auto import tqdm
import datasets

print(f"   numpy version: {np.__version__}")
print(f"   scipy version: {scipy.__version__}")

# ============================================================
# 3.2 DOWNLOAD REQUIRED MODELS
# ============================================================
print("\n📥 Downloading openWakeWord model files...")

model_dir = "./openwakeword/openwakeword/resources/models"
os.makedirs(model_dir, exist_ok=True)

model_files = [
    ("embedding_model.onnx", "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.onnx"),
    ("embedding_model.tflite", "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.tflite"),
    ("melspectrogram.onnx", "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.onnx"),
    ("melspectrogram.tflite", "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.tflite"),
]

for filename, url in model_files:
    filepath = os.path.join(model_dir, filename)
    if not os.path.exists(filepath):
        !wget -q -O {filepath} {url}
        print(f"   ✅ {filename}")
    else:
        print(f"   ⏭️ {filename} (already exists)")

# ============================================================
# 3.3 DOWNLOAD ROOM IMPULSE RESPONSES
# ============================================================
print("\n📥 Downloading room impulse responses...")

rir_dir = "./mit_rirs"
if not os.path.exists(rir_dir) or len(os.listdir(rir_dir)) < 100:
    os.makedirs(rir_dir, exist_ok=True)

    # Install git-lfs
    !git lfs install

    if not os.path.exists("MIT_environmental_impulse_responses"):
        !git clone -q https://huggingface.co/datasets/davidscripka/MIT_environmental_impulse_responses

    # Process the RIR files
    wav_files = list(Path("./MIT_environmental_impulse_responses/16khz").glob("*.wav"))
    if wav_files:
        rir_dataset = datasets.Dataset.from_dict({
            "audio": [str(i) for i in wav_files]
        }).cast_column("audio", datasets.Audio())

        for row in tqdm(rir_dataset, desc="Processing RIRs"):
            name = row['audio']['path'].split('/')[-1]
            scipy.io.wavfile.write(
                os.path.join(rir_dir, name),
                16000,
                (row['audio']['array'] * 32767).astype(np.int16)
            )
        print(f"   ✅ {len(os.listdir(rir_dir))} RIR files")
    else:
        print("   ⚠️ No RIR files found in cloned repo")
else:
    print(f"   ⏭️ RIRs already downloaded ({len(os.listdir(rir_dir))} files)")

# ============================================================
# 3.4 DOWNLOAD BACKGROUND AUDIO
# ============================================================
print("\n📥 Downloading background audio...")

# AudioSet - currently unavailable due to dataset restructuring
audioset_dir = "./audioset_16k"

if not os.path.exists(audioset_dir) or len([f for f in os.listdir(audioset_dir) if f.endswith('.wav')]) < 50:
    os.makedirs(audioset_dir, exist_ok=True)

    print("   ⏭️ Skipping AudioSet (dataset recently restructured)")
    print("   Using FMA + pre-computed features for background audio instead.")
else:
    count = len([f for f in os.listdir(audioset_dir) if f.endswith('.wav')])
    print(f"   ⏭️ AudioSet already downloaded ({count} files)")

# FMA (Free Music Archive)
fma_dir = "./fma"
if not os.path.exists(fma_dir) or len([f for f in os.listdir(fma_dir) if f.endswith('.wav')]) < 50:
    os.makedirs(fma_dir, exist_ok=True)
    print("   Loading FMA dataset (streaming)...")

    try:
        fma_dataset = datasets.load_dataset("rudraml/fma", name="small", split="train", streaming=True)
        fma_dataset = iter(fma_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000)))

        n_hours = 3  # 3 hours of music clips for better variety
        n_clips = n_hours * 3600 // 30  # FMA clips are 30 seconds each

        for i in tqdm(range(n_clips), desc="Processing FMA"):
            try:
                row = next(fma_dataset)
                name = row['audio']['path'].split('/')[-1].replace(".mp3", ".wav")
                scipy.io.wavfile.write(
                    os.path.join(fma_dir, name),
                    16000,
                    (row['audio']['array'] * 32767).astype(np.int16)
                )
            except StopIteration:
                break
            except Exception as e:
                continue  # Skip problematic files
        print(f"   ✅ {len([f for f in os.listdir(fma_dir) if f.endswith('.wav')])} FMA files")
    except Exception as e:
        print(f"   ⚠️ FMA download failed: {e}")
else:
    count = len([f for f in os.listdir(fma_dir) if f.endswith('.wav')])
    print(f"   ⏭️ FMA already downloaded ({count} files)")

# ============================================================
# 3.5 DOWNLOAD PRE-COMPUTED FEATURES
# ============================================================
print("\n📥 Downloading pre-computed features (this is the big download)...")

# Training features (~17GB)
features_file = "./openwakeword_features_ACAV100M_2000_hrs_16bit.npy"
if not os.path.exists(features_file):
    print("   ⬇️ Downloading training features (~17 GB)...")
    print("   This may take 10-30 minutes depending on connection speed.")
    !wget -q --show-progress https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/openwakeword_features_ACAV100M_2000_hrs_16bit.npy
    print("   ✅ Training features downloaded")
else:
    size_gb = os.path.getsize(features_file) / 1024 / 1024 / 1024
    print(f"   ⏭️ Training features already downloaded ({size_gb:.1f} GB)")

# Validation features (~176MB)
val_file = "validation_set_features.npy"
if not os.path.exists(val_file):
    print("   ⬇️ Downloading validation features (~176 MB)...")
    !wget -q --show-progress https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/validation_set_features.npy
    print("   ✅ Validation features downloaded")
else:
    size_mb = os.path.getsize(val_file) / 1024 / 1024
    print(f"   ⏭️ Validation features already downloaded ({size_mb:.0f} MB)")

# ============================================================
# VERIFICATION
# ============================================================
print("\n" + "="*60)
print("📊 VERIFICATION")
print("="*60)

def count_wav_files(directory):
    if os.path.isdir(directory):
        return len([f for f in os.listdir(directory) if f.endswith('.wav')])
    return 0

rir_count = count_wav_files(rir_dir)
audioset_count = count_wav_files(audioset_dir)
fma_count = count_wav_files(fma_dir)
total_bg = audioset_count + fma_count

checks = [
    ("RIRs", rir_count, 100),
    ("AudioSet", audioset_count, 0),  # Optional - 0 minimum
    ("FMA", fma_count, 50),
]

all_ok = True
for name, actual_count, min_count in checks:
    if min_count > 0:
        status = "✅" if actual_count >= min_count else "⚠️"
        print(f"   {status} {name}: {actual_count} files (need {min_count}+)")
        if actual_count < min_count:
            all_ok = False
    else:
        status = "✅" if actual_count > 0 else "⏭️"
        print(f"   {status} {name}: {actual_count} files (optional)")

# Check feature files
for name, path in [("Training features", features_file), ("Validation features", val_file)]:
    if os.path.exists(path):
        size = os.path.getsize(path)
        size_str = f"{size/1024/1024/1024:.1f} GB" if size > 1024*1024*1024 else f"{size/1024/1024:.0f} MB"
        print(f"   ✅ {name} ({size_str})")
    else:
        print(f"   ❌ {name} (missing)")
        all_ok = False

print(f"\n   Total background audio: {total_bg} files (need 50+)")

if total_bg < 50:
    all_ok = False

if all_ok:
    print("\n" + "="*60)
    print("✅ STEP 3 COMPLETE - All data downloaded!")
    print("="*60)
    print("\n👉 Proceed to Step 4 to train your model.")
else:
    print("\n" + "="*60)
    print("⚠️ Some downloads may have failed.")
    print("="*60)
    print("\nMinimum requirements:")
    print("   • 50+ total background audio files (AudioSet + FMA)")
    print("   • Training features file (17 GB)")
    print("   • Validation features file (176 MB)")
    print("\nRe-run this cell to retry, or proceed if you have enough background audio.")


📦 STEP 3: ENVIRONMENT SETUP & DATA DOWNLOAD

🔧 Installing dependencies...
openwakeword already cloned
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openwakeword 0.6.0 requires ai-edge-litert<3,>=2.0.2; platform_system == "Linux" or platform_system == "Darwin", which is not installed.
openwakeword 0.6.0 requires onnxruntime<2,>=1.10.0, which is not installed.
openwakeword 0.6.0 requires speexdsp-ns<1,>=0.1.2; platform_system == "Linux", which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openwakeword 0.6.0 requires speexdsp-ns<1,>=0.1.2; platform_system == "Linux", which is not installed.
openwakeword 0.6.0 requires ai-edge-litert<3,>=2.0.2; platform_system == "Linux" or platform_system == "Darwin", but you have ai-

/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: module 'pyarrow' has no attribute 'PyExtensionType'

In [ ]:
# @title ## 🚀 Step 4: Train Models { display-mode: "form" }
# @markdown This trains a model for each wake word you specified.
# @markdown
# @markdown **Time:** ~30-90 minutes per model (depends on settings and hardware)
# @markdown
# @markdown **Training phases:**
# @markdown 1. Generate synthetic speech clips
# @markdown 2. Augment clips with noise/reverb
# @markdown 3. Train neural network
# @markdown 4. Export to ONNX format
# @markdown
# @markdown ---
# @markdown ### 📁 Google Drive Settings (Recommended!)
# @markdown Colab's browser download can be unreliable. Google Drive ensures your models are saved safely.

enable_google_drive = True # @param {type:"boolean"}
# @markdown ↑ Enable to save models directly to Google Drive as soon as they finish.

drive_folder_name = "OpenWakeWord_Models" # @param {type:"string"}
# @markdown ↑ Folder name in your Google Drive (created automatically if it doesn't exist).

import yaml
import sys
import os
import re
import shutil

print("="*60)
print("🚀 STEP 4: MODEL TRAINING")
print("="*60)

# ============================================================
# GOOGLE DRIVE SETUP (if enabled)
# ============================================================
gdrive_enabled = False
gdrive_path = None

if enable_google_drive:
    print("\n☁️  Setting up Google Drive...")
    try:
        from google.colab import drive

        # Check if already mounted
        if not os.path.ismount('/content/drive'):
            print("   (You may be prompted to authorize access)\n")
            drive.mount('/content/drive')
        else:
            print("   Drive already mounted.")

        # Create the output folder
        drive_base = '/content/drive/MyDrive'
        drive_output_path = os.path.join(drive_base, drive_folder_name)

        if not os.path.exists(drive_output_path):
            os.makedirs(drive_output_path)
            print(f"   📂 Created folder: Google Drive/{drive_folder_name}/")
        else:
            print(f"   📂 Using folder: Google Drive/{drive_folder_name}/")

        # Verify write access
        test_file = os.path.join(drive_output_path, '.test_write')
        with open(test_file, 'w') as f:
            f.write('test')
        os.remove(test_file)

        gdrive_enabled = True
        gdrive_path = drive_output_path
        print(f"   ✅ Google Drive connected! Models will be saved there.")

    except ImportError:
        print("   ⚠️ Google Drive only available in Colab. Using local storage.")
    except Exception as e:
        print(f"   ⚠️ Drive setup failed: {e}")
        print("   Models will be downloaded via browser instead.")
else:
    print("\n💾 Google Drive: DISABLED")
    print("   Models will be downloaded via browser after training.")
    print("   ⚠️ Note: Browser downloads can be unreliable in Colab.")

def sanitize_name(name):
    """Convert wake word to valid filename."""
    return re.sub(r'[^a-zA-Z0-9]+', '_', name).strip('_')

def save_to_drive(onnx_path, model_name):
    """Copy model to Google Drive. Returns True if successful."""
    if not gdrive_enabled or not gdrive_path:
        return False

    try:
        dest_path = os.path.join(gdrive_path, f"{model_name}.onnx")
        shutil.copy2(onnx_path, dest_path)

        # Verify the copy
        if os.path.exists(dest_path):
            size_kb = os.path.getsize(dest_path) / 1024
            print(f"\n☁️  SAVED TO GOOGLE DRIVE: {model_name}.onnx ({size_kb:.1f} KB)")
            print(f"   Location: Google Drive/{drive_folder_name}/{model_name}.onnx")
            return True
        else:
            print(f"\n⚠️  Drive copy verification failed for {model_name}")
            return False
    except Exception as e:
        print(f"\n⚠️  Failed to save to Drive: {e}")
        return False

def queue_download(onnx_path, model_name):
    """Queue a model file for browser download (Colab only). Non-blocking."""
    try:
        from google.colab import files
        import threading

        def trigger_download():
            try:
                files.download(onnx_path)
            except:
                pass  # Ignore errors in background thread

        print(f"\n⬇️  Queued {model_name}.onnx for download")
        thread = threading.Thread(target=trigger_download)
        thread.daemon = True
        thread.start()

        import time
        time.sleep(1)
        return True
    except ImportError:
        print(f"\n📁 Not running in Colab - find your model at: {onnx_path}")
        return False
    except Exception as e:
        print(f"\n⚠️  Auto-download skipped: {e}")
        return False

# ============================================================
# LOAD CONFIG AND START TRAINING
# ============================================================
print("\n" + "="*60)
print("🎯 STARTING TRAINING")
print("="*60)

base_config = yaml.load(
    open("openwakeword/examples/custom_model.yml", 'r').read(),
    yaml.Loader
)

output_dir = "./my_custom_model"
os.makedirs(output_dir, exist_ok=True)

successful_models = []
failed_models = []
models_saved_to_drive = []
models_pending_download = []

for i, word in enumerate(wake_word_list):
    model_name = sanitize_name(word)

    print(f"\n{'='*60}")
    print(f"🎯 TRAINING MODEL {i+1}/{len(wake_word_list)}: '{word}'")
    print(f"   Model name: {model_name}")
    print(f"{'='*60}")

    # Create config for this word
    config = base_config.copy()
    config["target_phrase"] = [word]
    config["model_name"] = model_name
    config["n_samples"] = number_of_examples
    config["n_samples_val"] = max(500, number_of_examples // 10)
    config["steps"] = number_of_training_steps
    config["target_accuracy"] = target_accuracy
    config["target_recall"] = target_recall
    config["target_false_positives_per_hour"] = target_false_positives_per_hour
    config["output_dir"] = output_dir
    config["max_negative_weight"] = false_activation_penalty
    config["layer_size"] = layer_size
    config["background_paths"] = ['./audioset_16k', './fma']
    config["false_positive_validation_data_path"] = "validation_set_features.npy"
    config["feature_data_files"] = {"ACAV100M_sample": "openwakeword_features_ACAV100M_2000_hrs_16bit.npy"}

    config_file = f'{model_name}_config.yaml'
    with open(config_file, 'w') as f:
        yaml.dump(config, f)

    try:
        # Phase 1: Generate clips
        print(f"\n📝 Phase 1/3: Generating {number_of_examples:,} synthetic speech clips...")
        !{sys.executable} openwakeword/openwakeword/train.py --training_config {config_file} --generate_clips

        # Phase 2: Augment clips
        print(f"\n🔊 Phase 2/3: Augmenting clips with noise and reverb...")
        !{sys.executable} openwakeword/openwakeword/train.py --training_config {config_file} --augment_clips

        # Phase 3: Train model
        print(f"\n🧠 Phase 3/3: Training neural network ({number_of_training_steps:,} steps)...")
        !{sys.executable} openwakeword/openwakeword/train.py --training_config {config_file} --train_model

        # Check if ONNX was created
        onnx_path = f"{output_dir}/{model_name}.onnx"
        if os.path.exists(onnx_path):
            size_kb = os.path.getsize(onnx_path) / 1024
            print(f"\n✅ SUCCESS: {onnx_path} ({size_kb:.1f} KB)")

            model_info = {
                'word': word,
                'model_name': model_name,
                'onnx_path': onnx_path
            }
            successful_models.append(model_info)

            # Try to save to Google Drive first
            if gdrive_enabled and save_to_drive(onnx_path, model_name):
                models_saved_to_drive.append(model_info)
            else:
                # Fall back to queuing download (if not using Drive)
                models_pending_download.append(model_info)
        else:
            print(f"\n❌ ONNX model not found at {onnx_path}")
            failed_models.append({'word': word, 'error': 'ONNX not created'})

    except Exception as e:
        print(f"\n❌ Training failed: {e}")
        failed_models.append({'word': word, 'error': str(e)})

# ============================================================
# SUMMARY
# ============================================================
print(f"\n{'='*60}")
print(f"📊 TRAINING SUMMARY")
print(f"{'='*60}")
print(f"\n✅ Successful: {len(successful_models)}")
for m in successful_models:
    print(f"   • {m['word']} → {m['onnx_path']}")

if failed_models:
    print(f"\n❌ Failed: {len(failed_models)}")
    for m in failed_models:
        print(f"   • {m['word']}: {m['error']}")

# Report on saving method
if models_saved_to_drive:
    print(f"\n☁️  SAVED TO GOOGLE DRIVE: {len(models_saved_to_drive)} model(s)")
    print(f"   Location: Google Drive/{drive_folder_name}/")
    for m in models_saved_to_drive:
        print(f"   • {m['model_name']}.onnx")
    print(f"\n✨ Your models are safely stored in Google Drive!")
    print(f"   You can access them anytime, even after this session ends.")

if models_pending_download:
    print(f"\n⬇️  PENDING DOWNLOAD: {len(models_pending_download)} model(s)")
    print(f"   Run Step 5 to download these models.")

if not successful_models:
    print(f"\n⚠️ No models were trained successfully. Check the errors above.")

In [ ]:
# @title ## ⬇️ Step 5: Download Your Models { display-mode: "form" }
# @markdown Downloads all generated model files via browser.
# @markdown
# @markdown **Note:** If you enabled Google Drive in Step 4, your models are already saved there!
# @markdown This step is mainly for backup or if you disabled Google Drive.

import os
import shutil
from datetime import datetime

print("="*60)
print("⬇️ STEP 5: DOWNLOAD MODELS")
print("="*60)

# Check for variables from Step 4
try:
    _gdrive_enabled = gdrive_enabled
    _gdrive_path = gdrive_path
    _drive_folder = drive_folder_name
except NameError:
    _gdrive_enabled = False
    _gdrive_path = None
    _drive_folder = "OpenWakeWord_Models"

try:
    models_to_download = successful_models
except NameError:
    models_to_download = []

if not models_to_download:
    print("\n⚠️ No models to download. Run Step 4 first.")
else:
    # Show Google Drive status
    if _gdrive_enabled and _gdrive_path:
        print(f"\n☁️  Google Drive Status: CONNECTED")
        print(f"   Your models are already saved to:")
        print(f"   Google Drive/{_drive_folder}/")
        print(f"\n   The download below is a backup copy.")

    print(f"\n📦 Generated Models:\n")
    print(f"{'Model':<35} {'Size':<15} {'Drive Status'}")
    print(f"{'-'*65}")

    download_files = []
    output_dir = "./my_custom_model"

    for m in models_to_download:
        model_name = m['model_name']
        onnx_path = m.get('onnx_path', f"{output_dir}/{model_name}.onnx")

        # Check local file
        if os.path.exists(onnx_path):
            size_kb = os.path.getsize(onnx_path) / 1024

            # Check if it's in Drive
            drive_status = "—"
            if _gdrive_enabled and _gdrive_path:
                drive_file = os.path.join(_gdrive_path, f"{model_name}.onnx")
                if os.path.exists(drive_file):
                    drive_status = "✅ Saved"
                else:
                    drive_status = "❌ Missing"

            print(f"{model_name}.onnx{' '*(30-len(model_name))} {size_kb:.1f} KB{' '*(10-len(f'{size_kb:.1f}'))} {drive_status}")
            download_files.append(onnx_path)
        else:
            print(f"{model_name}.onnx{' '*(30-len(model_name))} ❌ not found")

    # Offer to save missing files to Drive
    if _gdrive_enabled and _gdrive_path:
        missing_from_drive = []
        for m in models_to_download:
            model_name = m['model_name']
            onnx_path = m.get('onnx_path', f"{output_dir}/{model_name}.onnx")
            drive_file = os.path.join(_gdrive_path, f"{model_name}.onnx")
            if os.path.exists(onnx_path) and not os.path.exists(drive_file):
                missing_from_drive.append((onnx_path, model_name))

        if missing_from_drive:
            print(f"\n📤 Copying {len(missing_from_drive)} missing model(s) to Google Drive...")
            for onnx_path, model_name in missing_from_drive:
                try:
                    dest_path = os.path.join(_gdrive_path, f"{model_name}.onnx")
                    shutil.copy2(onnx_path, dest_path)
                    print(f"   ✅ {model_name}.onnx")
                except Exception as e:
                    print(f"   ❌ {model_name}.onnx - {e}")

    # Create zip archive and trigger download
    if download_files:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        zip_name = f'openwakeword_models_{timestamp}'

        # Copy files to temp directory for zipping
        os.makedirs(zip_name, exist_ok=True)
        for f in download_files:
            shutil.copy(f, zip_name)

        shutil.make_archive(zip_name, 'zip', zip_name)
        zip_path = f'{zip_name}.zip'

        print(f"\n📁 Created archive: {zip_path}")
        print(f"   Size: {os.path.getsize(zip_path) / 1024:.1f} KB")

        # Auto-download in Colab
        try:
            from google.colab import files
            print("\n⬇️ Starting download...")
            print("   (If download doesn't start, check your browser's download folder)")

            # Download zip first
            files.download(zip_path)

            # Also offer individual files
            print("\n📥 Individual file downloads:")
            for f in download_files:
                try:
                    files.download(f)
                    print(f"   ✅ {os.path.basename(f)}")
                except:
                    print(f"   ⚠️ {os.path.basename(f)} - download may have failed")

        except ImportError:
            print(f"\n📥 Download manually from the file browser on the left.")

        # Cleanup temp dir
        shutil.rmtree(zip_name, ignore_errors=True)

        if _gdrive_enabled:
            print(f"\n💡 Remember: Your models are also in Google Drive!")
            print(f"   Google Drive/{_drive_folder}/")
    else:
        print("\n⚠️ No model files found to download.")

In [ ]:
# @title ## 🧪 Step 6 (Optional): Test Your Models { display-mode: "form" }
# @markdown Quick sanity check that your models load and run.

import numpy as np
import os

print("="*60)
print("🧪 STEP 6: MODEL TESTING")
print("="*60)

try:
    models_to_test = successful_models
except NameError:
    models_to_test = []

output_dir = "./my_custom_model"

if not models_to_test:
    print("\n⚠️ No models to test. Run Step 4 first.")
else:
    try:
        import openwakeword
        from openwakeword.model import Model

        for m in models_to_test:
            model_name = m['model_name']
            word = m['word']
            onnx_path = m.get('onnx_path', f"{output_dir}/{model_name}.onnx")

            print(f"\n📊 Testing: {word} ({model_name})")

            if os.path.exists(onnx_path):
                try:
                    model = Model(
                        wakeword_models=[onnx_path],
                        inference_framework='onnx'
                    )

                    # Test with silence (should not trigger)
                    test_audio = np.zeros(16000, dtype=np.int16)
                    prediction = model.predict(test_audio)

                    print(f"   ✅ Model loaded successfully")
                    print(f"   Prediction on silence: {prediction}")
                    print(f"   (Should be close to 0.0 - no wake word in silence)")

                except Exception as e:
                    print(f"   ❌ Error testing model: {e}")
            else:
                print(f"   ⚠️ Model file not found: {onnx_path}")

    except ImportError as e:
        print(f"\n⚠️ Could not import openwakeword: {e}")
        print("   Models were still created - you can test them in your own environment.")

---

## 📖 How to Use Your Models

### Home Assistant

1. Download your `.onnx` model file (from Google Drive or Step 5)
2. Copy to your Home Assistant config: `/config/openwakeword/`
3. In the openWakeWord add-on settings, add your custom model path
4. Restart the add-on

See: [Home Assistant openWakeWord docs](https://github.com/home-assistant/addons/blob/master/openwakeword/DOCS.md#custom-wake-word-models)

### Python

```python
from openwakeword.model import Model
import pyaudio
import numpy as np

# Load your model
model = Model(wakeword_models=['path/to/your_model.onnx'])

# Setup audio stream
pa = pyaudio.PyAudio()
stream = pa.open(
    rate=16000,
    channels=1,
    format=pyaudio.paInt16,
    input=True,
    frames_per_buffer=1280
)

# Listen for wake word
print("Listening for wake word...")
while True:
    audio = np.frombuffer(stream.read(1280), dtype=np.int16)
    prediction = model.predict(audio)
    
    for model_name, score in prediction.items():
        if score > 0.5:  # Adjust threshold as needed
            print(f"Wake word detected! Score: {score:.3f}")
```

---

## 🔧 Troubleshooting

### Model doesn't detect well
- Try different phonetic spellings in Step 1
- Increase `number_of_examples` to 40,000+
- Increase `number_of_training_steps` to 40,000+
- Lower the detection threshold (e.g., 0.3 instead of 0.5)

### Too many false activations
- Increase `false_activation_penalty` (try 2000-3000)
- Lower `target_false_positives_per_hour` (try 0.1)
- Raise the detection threshold (e.g., 0.7 instead of 0.5)

### Training times out
- Reduce `number_of_examples` to 10,000-20,000
- Reduce `number_of_training_steps` to 10,000-20,000
- Use Google Colab Pro for longer runtimes
- Use GPU runtime (faster training)

### Download hangs or fails
- **Enable Google Drive in Step 4!** This provides reliable model saving
- Models are copied to Drive immediately after each one completes
- Even if Colab disconnects, your models are safe in Drive
- Re-run Step 5 to retry browser downloads if needed

### "ModuleNotFoundError" during training
- This is usually harmless - check if the `.onnx` file was still created
- The training script may show errors about optional components

### "RecursionError" in Step 1
- This was a bug in the torch.load patching - now fixed!
- If you still see it, restart the runtime and try again

---

## 📚 Resources

- [openWakeWord GitHub](https://github.com/dscripka/openWakeWord)
- [openWakeWord Documentation](https://github.com/dscripka/openWakeWord/blob/main/docs/)
- [Home Assistant Voice](https://www.home-assistant.io/voice_control/)
- [Piper TTS](https://github.com/rhasspy/piper) (used for synthetic speech)